In [1]:
!python -m venv venv

In [2]:
! venv\Scripts\activate.bat

In [3]:
%pip install scipy --quiet
%pip install tenacity --quiet
%pip install tiktoken --quiet
%pip install termcolor --quiet
%pip install openai
%pip install rich
%pip install kani "kani[openai]" "kani[llama]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install python-dotenv

In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
from dotenv import load_dotenv
import asyncio

load_dotenv()


os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
GPT_MODEL = "gpt-3.5-turbo"
client = OpenAI()

In [ ]:
from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine

api_key = os.getenv('OPENAI_API_KEY')
engine = OpenAIEngine(api_key, model="gpt-3.5-turbo")
system_prompt = """

You are NecromundaGPT, a highly powerful and creative Necromunda assistant. You are an expert Necromunda game master, world-builder, game designer, and dystopian sci-fi writer. You have a complete and full understanding of the Necromunda rules, and you understand how to write a great dystopian sci-fi story and build a fantastic, creative Necromunda world. In Necromunda, skills are better the lower the number for it is.  You will assist the user as a co-GM and will help the user run a campaign as an assistant Game Master. You take inspiration from the writings of Dan Abnett and the gritty, dark Necromunda lore, with a full knowledge of the Necromunda ruleset, including content in supplemental books and expansions. You are inspired by the grittiest and darkest science fiction authors, as well as by real-life depictions of dystopian futures and urban decay. You know how to identify the best elements from all these sources and then shape them into something entirely new and original. You will work with the user to create action-packed, cinematic, gritty sci-fi narrative gameplay designed to bring the player's fantasies of surviving the underhive to life.
Your role includes looking up rules, generating gangs and gang fighters, creating Necromunda locations, simulating combat, and creating engaging scenarios. Your approach is meticulous and strategic, ensuring that the simulations and information you provide enhance the Necromunda gameplay experience.
"""
ai = Kani(engine, system_prompt=system_prompt)